In [1]:
!pip install tensorflow[and-cuda] scikit-learn deep-translator nltk fastapi matplotlib

In [2]:
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Embedding, Bidirectional, Dropout, LSTM  # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer  # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences  # type: ignore
from tensorflow.keras.models import Sequential  # type: ignore
import matplotlib.pyplot as plt
import pandas as pd
import re
import pickle

2024-07-17 13:21:58.191838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 13:21:58.443775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 13:21:58.520994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 13:21:58.696996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 13:22:09.419323: W tensorflow/compiler/tf2

In [3]:
# Define the output path for the negative reviews CSV
output_path_neg = "./negative.csv"

# Define the path to the dataset
ds_path = "./Hotel_Reviews.csv"
 
# Remove the file if it already exists
if os.path.exists(output_path_neg):
    os.remove(output_path_neg)

# Read the data from Hotel_Reviews.csv
reviews_df = pd.read_csv(ds_path)

# Clean up whitespace in "Negative_Review" column
reviews_df["Negative_Review"] = reviews_df["Negative_Review"].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

# Define list of phrases to filter out
phrases_to_filter = ["no negative", "no negatives", "all ok", "nothing", "n a", "no problem", "no problems", "none", "i loved everything", "no bad experiences", ""]

# Filter out rows with specific phrases in "Negative_Review" (case insensitive)
reviews_df = reviews_df[~reviews_df["Negative_Review"].str.lower().isin(phrases_to_filter)]

# Append the cleaned negative text reviews to a new "review" column
reviews_df["review"] = reviews_df["Negative_Review"]

# Create the "sentiment" column with value 0 (negative sentiment)
reviews_df["sentiment"] = 0

# Select only the "review" and "sentiment" columns
reviews_df = reviews_df[["review", "sentiment"]]

# Save the new DataFrame as a CSV
reviews_df.to_csv(output_path_neg, index=False)

# Print confirmation message
print(f"Generated negative.csv: {output_path_neg}")

Generated negative.csv: ./negative.csv


In [4]:
# Define the output path for the positive reviews CSV
output_path_pos = "./positive.csv"

# Define the path to the dataset
ds_path = "./Hotel_Reviews.csv"
 
# Remove the file if it already exists
if os.path.exists(output_path_pos):
    os.remove(output_path_pos)

# Read the data from Hotel_Reviews.csv
reviews_df = pd.read_csv(ds_path)

# Clean up whitespace in "Positive_Review" column
reviews_df["Positive_Review"] = reviews_df["Positive_Review"].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

# Define list of phrases to filter out (if needed for positive reviews)
# For positive reviews, you might filter out phrases like "no positive", "none", etc.
phrases_to_filter = ["no positive", "no positives", "none", ""]

# Filter out rows with specific phrases in "Positive_Review" (case insensitive)
reviews_df = reviews_df[~reviews_df["Positive_Review"].str.lower().isin(phrases_to_filter)]

# Append the cleaned positive text reviews to a new "review" column
reviews_df["review"] = reviews_df["Positive_Review"]

# Create the "sentiment" column with value 1 (positive sentiment)
reviews_df["sentiment"] = 1

# Select only the "review" and "sentiment" columns
reviews_df = reviews_df[["review", "sentiment"]]

# Save the new DataFrame as a CSV
reviews_df.to_csv(output_path_pos, index=False)

# Print confirmation message
print(f"Generated positive.csv: {output_path_pos}")

Generated positive.csv: ./positive.csv


In [5]:
# Define the paths for the input CSV files
first_csv_path = "./negative.csv"
second_csv_path = "./positive.csv"
output_path = "./dataset.csv"

# Read the data from the CSV files
first_df = pd.read_csv(first_csv_path)
second_df = pd.read_csv(second_csv_path)

# Concatenate the DataFrames
combined_df = pd.concat([first_df, second_df], ignore_index=True)

# Delete the output file if it exists
if os.path.exists(output_path):
    os.remove(output_path)

# Save the combined DataFrame as a new CSV
combined_df.to_csv(output_path, index=False)

print(f"Combined negative and positive file saved at: {output_path}")

Combined negative and positive file saved at: ./dataset.csv


In [6]:
# import pandas as pd
# import os

# # Define the path for the new CSV file
# output_path = "./second.csv"

# # Delete the file if it exists
# if os.path.exists(output_path):
#     os.remove(output_path)

# # Read the data
# reviews_df = pd.read_csv("./tripadvisor_hotel_reviews.csv")

# # Convert ratings to binary 1-0 scale for is_bad_review
# def convert_rating(rating):
#     if rating <= 2:
#         return 0  # Bad review
#     elif rating >= 4:
#         return 1  # Good review
#     else:
#         return None  # Ignore neutral reviews with a rating of 3

# reviews_df["sentiment"] = reviews_df["Rating"].apply(convert_rating)

# # Drop neutral reviews
# reviews_df = reviews_df.dropna(subset=["sentiment"])

# # Select only relevant columns
# reviews_df = reviews_df[["Review", "sentiment"]]
# reviews_df.columns = ["review", "sentiment"]

# # Convert sentiment to integer type
# reviews_df["sentiment"] = reviews_df["sentiment"].astype(int)

# # Save the new DataFrame as a CSV
# reviews_df.to_csv(output_path, index=False)

# print(f"New CSV file saved at: {output_path}")

# # 

In [7]:
# import pandas as pd
# import os

# # Define the paths for the input CSV files
# first_csv_path = "./first.csv"
# second_csv_path = "./second.csv"
# output_path = "./dataset.csv"

# # Read the data from the CSV files
# first_df = pd.read_csv(first_csv_path)
# second_df = pd.read_csv(second_csv_path)

# # Concatenate the DataFrames
# combined_df = pd.concat([first_df, second_df], ignore_index=True)

# # Delete the output file if it exists
# if os.path.exists(output_path):
#     os.remove(output_path)

# # Save the combined DataFrame as a new CSV
# combined_df.to_csv(output_path, index=False)

# print(f"Combined CSV file saved at: {output_path}")

In [8]:
TAG_RE = re.compile(r"<[^>]+>")
STOPWORDS = set(stopwords.words("english"))
PUNCTUATIONS = re.compile("[^a-zA-Z]")


def preprocess_text(sen):
    # Lowercase the text
    sentence = sen.lower()
    # Remove HTML tags
    sentence = TAG_RE.sub("", sentence)
    # Remove punctuations
    sentence = PUNCTUATIONS.sub(" ", sentence)
    # Remove single characters
    sentence = re.sub(r"\s+[a-zA-Z]\s+", " ", sentence)
    # Remove multiple spaces
    sentence = re.sub(r"\s+", " ", sentence)
    # Remove stopwords
    sentence = " ".join(word for word in sentence.split() if word not in STOPWORDS)
    return sentence

In [9]:
# Load the CSV file
reviews = pd.read_csv("./dataset.csv")
reviews

,review,sentiment
0,I am so angry that i made this post available ...,0
1,Rooms are nice but for elderly a bit difficult...,0
2,My room was dirty and I was afraid to walk bar...,0
3,You When I booked with your company on line yo...,0
4,Backyard of the hotel is total mess shouldn t ...,0
...,...,...
292579,Good location Nice big room,1
292580,I loved the style of the hotel and the locatio...,1
292581,With a Subway station a few steps away it s ve...,1
292582,Room was clean nice as advertised,1


In [10]:
def preprocess_text(sen):
    
    # Lowercase the text
    sentence = sen.lower()
    # Remove HTML tags
    sentence = TAG_RE.sub("", sentence)
    # Remove punctuations
    sentence = PUNCTUATIONS.sub(" ", sentence)
    # Remove single characters
    sentence = re.sub(r"\s+[a-zA-Z]\s+", " ", sentence)
    # Remove multiple spaces
    sentence = re.sub(r"\s+", " ", sentence)
    # Remove stopwords
    sentence = " ".join(word for word in sentence.split() if word not in STOPWORDS)
    return sentence

In [11]:
# Load the CSV file
reviews = pd.read_csv("./dataset.csv")

# Preprocess each review and store in a list of dictionaries
data = []
for i, row in reviews.iterrows():
    # Check if the text is NaN
    if isinstance(row["review"], str):
        review = preprocess_text(row["review"])
        data.append({"org": row["review"], "review": review, "sentiment": row["sentiment"]})
        
# Create DataFrame from the preprocessed data
X = pd.DataFrame(data)

In [12]:
# Split the data into training (80%) and validation (20%)
train_data, val_data = train_test_split(X, test_size=0.2, random_state=42)

# Create a TensorFlow dataset from the training data
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_data["review"].values, train_data["sentiment"].values)
)

# Shuffle and batch the training dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(train_data)).batch(batch_size)

# Repeat the process for the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_data["review"].values, val_data["sentiment"].values)
)
val_dataset = val_dataset.batch(batch_size)

# Print the shapes of the datasets
print("Train data shape:", len(train_data))
print("Validation data shape:", len(val_data))

I0000 00:00:1721194123.266826   21751 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721194123.723969   21751 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721194123.726610   21751 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721194123.741844   21751 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Train data shape: 234021
Validation data shape: 58506


In [13]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data["review"].values)

with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert text data to sequences
X_train = tokenizer.texts_to_sequences(train_data["review"].values)
X_val = tokenizer.texts_to_sequences(val_data["review"].values)

# Pad sequences to ensure uniform length
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len, padding="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="post")
vocab_size = len(tokenizer.word_index) + 1

In [14]:
def create_model():

    # Define the model
    model = Sequential()
    model.add(Input(shape=(max_len,)))
    model.add(Embedding(input_dim=vocab_size, output_dim=100))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

    # Train the model

    history = model.fit(
        X_train,
        train_data["sentiment"].values,
        epochs=10,
        batch_size=32,
        validation_data=(X_val, val_data["sentiment"].values),
        verbose=1,
    )

    # Plot training history

    plt.plot(history.history["accuracy"], label="accuracy")
    plt.plot(history.history["val_accuracy"], label="val_accuracy")

    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    return model

In [15]:
# Path to your model file
model_path = "./sentiment.h5"

try:
    # Attempt to load the model
    model = tf.keras.models.load_model(model_path)
    print("Loaded model from disk")
except (OSError, IOError) as e:
    # If loading fails, create a new model and save it
    print(f"Model file {model_path} not found. Creating a new model.")
    model = create_model()
    model.save(model_path)
    print(f"New model saved to {model_path}")

# Print model summary
model.summary()

Loaded model from disk


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │     3,684,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       160,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,845,603 (14.67 MB)

 Trainable params: 3,845,601 (14.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_val, val_data["sentiment"].values)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

 489/1829 ━━━━━━━━━━━━━━━━━━━━ 11:54 533ms/step - accuracy: 0.9373 - loss: 0.2041

In [ ]:
def predict_sentiment(review):
    # Tokenize and pad the input review
    review_seq = tokenizer.texts_to_sequences([preprocess_text(review)])
    review_seq = pad_sequences(review_seq, maxlen=max_len, padding="post")

    # Make prediction
    prediction = round(model.predict(review_seq)[0][0], 2)

    # Return the prediction
    return prediction

In [ ]:
from deep_translator import GoogleTranslator

review = "I love the resort"

 # Translate the input review to English
translatedText = GoogleTranslator(source="auto", target="en").translate(review)

result = predict_sentiment(translatedText)
result